In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark= SparkSession.builder.appName("trip").getOrCreate()

In [4]:
directory='c://Users//apfhd//SparkExam//data//fhvhv_tripdata_2020-03.csv'

In [8]:
# 컬럼정보 넣기 (header = True) 
datas=spark.read.csv(f'file:///{directory}',header=True)
datas.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [9]:
# SQL 사용 할 수 있게 임시 view등록
datas.createOrReplaceTempView("mobility_data")

In [10]:
query = '''
select * from mobility_data limit 5
'''
spark.sql(query).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [18]:
# 승차 년 월 일 별로 카운트 세기
query= '''
select split(pickup_datetime,' ')[0] as pickup_date, count(*) as trips
from mobility_data
group by pickup_date
'''
spark.sql(query).show(5)

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-02|648986|
| 2020-03-01|784246|
| 2020-03-03|697880|
| 2020-03-04|707879|
| 2020-03-05|731165|
+-----------+------+
only showing top 5 rows



##### 좀 더 좋은 방안
```
query= '''
select split(pickup_datetime,' ')[0] as pickup_date, count(*) as trips
from mobility_data
group by pickup_date
'''
```
-> 이 내용을 subquery로 활용!

In [25]:
# 좀 더 나은 방법
query= """
select pickup_date, count(*) as trips

from(select split(pickup_datetime,' ')[0] as pickup_date
from mobility_data)

group by pickup_date
"""
spark.sql(query).show(5)

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-02|648986|
| 2020-03-01|784246|
| 2020-03-03|697880|
| 2020-03-04|707879|
| 2020-03-05|731165|
+-----------+------+
only showing top 5 rows



In [26]:
# 실행 계획 살펴보기
spark.sql(query).explain(True)
# Parsed Logical Plan -> 작성 한 쿼리를 분석하는 단계


== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['pickup_date, 'count(1) AS trips#442]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['split('pickup_datetime,  )[0] AS pickup_date#441]
      +- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [pickup_date#441], [pickup_date#441, count(1) AS trips#442L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(pickup_datetime#180,  , -1)[0] AS pickup_date#441]
      +- SubqueryAlias mobility_data
         +- View (`mobility_data`, [hvfhs_license_num#178,dispatching_base_num#179,pickup_datetime#180,dropoff_datetime#181,PULocationID#182,DOLocationID#183,SR_Flag#184])
            +- Relation [hvfhs_license_num#178,dispatching_base_num#179,pickup_datetime#180,dropoff_datetime#181,PULocationID#182,DOLocationID#183,SR_Flag#184] csv

== Optimized Logical Plan ==
Aggregate [pickup_date#441], [pickup_date#441, count(1) AS trips#442L]
+- Proj

In [29]:
zone_file = "c://Users//apfhd//SparkExam//data//taxi+_zone_lookup.csv"
zone_data = spark.read.csv(f"file:///{zone_file}", inferSchema=True, header=True)
zone_data.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [30]:
zone_data.createOrReplaceTempView('zone_data')

In [31]:
query = '''

select borough, count(*) as trips

from(select zone_data.Borough as borough
    from mobility_data
    join zone_data on mobility_data.PULocationID = zone_data.LocationID)
    
group by borough

'''
spark.sql(query).show()

+-------------+-------+
|      borough|  trips|
+-------------+-------+
|       Queens|2437383|
|          EWR|    362|
|      Unknown|    845|
|     Brooklyn|3735764|
|Staten Island| 178818|
|    Manhattan|4953140|
|        Bronx|2086592|
+-------------+-------+



In [32]:
# 지역별 운영 횟수 - 그중에서도 HV003에 대한
query = """

select borough, count(*) as trips

from (select zone_data.Borough as borough
      from mobility_data
      join zone_data on mobility_data.PULocationID = zone_data.LocationID
      where mobility_data.hvfhs_license_num='HV0003'
      )

group by borough

"""

spark.sql(query).show()

+-------------+-------+
|      borough|  trips|
+-------------+-------+
|       Queens|1863688|
|      Unknown|    548|
|     Brooklyn|2779375|
|Staten Island| 148199|
|    Manhattan|3270666|
|        Bronx|1774283|
|          EWR|      4|
+-------------+-------+



In [33]:
spark.stop()